## Collect a Company's Patent Porfolio

This notebook will attempt to amass, for any given company, their entire worldwide patent porfolio.\*

### U.S. Portfolio

Collecting their U.S. portfolio will proceed in a few steps:

1. Collect all applications ever assigned to the company

    a. Those applications naming the company as an applicant
    b. Those assigned to the company via an assignment
    
2. Prune all applications assigned out of the company, as recorded in USPTO assignment records

\*NOTE: I say "attempt" because the notebook relies on the USPTO system, and the International Patent Document (INPADOC) service. Not all foreign jurisdictions participate in INPADOC, and not all INPADOC member states are as good at contributing to the system as we would like. So coverage is good, but not perfect.

First, we do our standard imports, and grab the USApplication and Assignment objects from patent_client

In [1]:
import pandas as pd
from patent_client import USApplication, Assignment

company_name = 'Tesla Motors'

### Step 1.a: Collect all applications naming the company as applicant

In [2]:
applicant_apps = USApplication.objects.filter(first_named_applicant=company_name).values_list('appl_id', flat=True).to_set()

### Step 1.b: Collect all applications assigned to the company

In [3]:
# The assigned apps is either a single value, or a list of values if more than one property was assigned
assigned_apps = Assignment.objects.filter(assignee=company_name).explode('properties').values_list('appl_id', flat=True).to_set()

And now we combine the two

In [4]:
all_apps = applicant_apps | assigned_apps

print(f'Total U.S. Applications Ever Owned by {company_name}: {len(all_apps)}')

Total U.S. Applications Ever Owned by Tesla Motors: 502


### Step 2: Collect all applications ever assigned out of the company

In [5]:
assigned_out_apps = set(Assignment.objects.filter(assignor=company_name).to_pandas()
                   .query('conveyance_text == "ASSIGNMENT OF ASSIGNORS INTEREST"')
                   .properties.explode().apply(lambda x: x.appl_id).to_list())
print(f'U.S. Applications assigned out of {company_name}: {len(assigned_out_apps)}')

U.S. Applications assigned out of Tesla Motors: 4


### Step 3: Subtract one set from the other

In [6]:
owned_apps = list(set(all_apps) - set(assigned_out_apps))
len(owned_apps)


498

### Step 4: Generate status report

In [7]:
import pandas as pd
from patent_client.parser import parse

us_df = USApplication.objects.filter(appl_id=owned_apps).values(
    'appl_id', 
    'app_filing_date', 
    'app_early_pub_number',
    'app_early_pub_date',
    'patent_number',
    'patent_issue_date',
    'patent_title',
    'inventors__0__name',
    'applicants__0__name',
    'app_status',
    'app_status_date'
).to_pandas()

us_df.head()

,appl_id,app_filing_date,app_early_pub_number,app_early_pub_date,patent_number,patent_issue_date,patent_title,inventors__0__name,applicants__0__name,app_status,app_status_date
0,15300044,2016-09-28,US20170177048A1,2017-06-22,None,None,FUNCTIONAL REDUNDANCY OF COMMUNICATIONS AND DA...,None,None,Response to Non-Final Office Action Entered an...,2020-02-22
1,15632587,2017-06-26,US20170291482A1,2017-10-12,None,None,INTEGRATED ELECTRIC MOTOR ASSEMBLY,None,None,Response after Final Action Forwarded to Examiner,2020-02-14
2,16202865,2018-11-28,US20190097508A1,2019-03-28,None,None,GEOMETRY OF ROTOR END RING AND STATOR END TURNS,None,None,Response to Non-Final Office Action Entered an...,2020-02-09
3,16382614,2019-04-12,US20190232344A1,2019-08-01,None,None,CLOSED-LOOP THERMAL SERVICING OF SOLVENT-REFIN...,None,None,Response to Non-Final Office Action Entered an...,2020-02-03
4,15849831,2017-12-21,US20180111576A1,2018-04-26,None,None,LOAD LIMITER CONFIGURATION USING MULTIDIMENSIO...,None,None,Notice of Allowance Mailed -- Application Rece...,2020-01-27


### Foreign Patent Portfolio

Now we will fetch the company's wordwide portfolio, as it appears in the Inpadoc system maintained by the EPO

In [8]:
from patent_client_plus import Inpadoc

foreign = Inpadoc.objects.filter(applicant=company_name)

And now, a status report:

In [9]:
foreign_df = foreign[:15].values(
    'country', 
    appl_id='biblio.applications.0.number', 
    app_filing_date='biblio.applications.0.date', 
    pub_number='biblio.publications.0.number', 
    pub_date='biblio.publications.0.date', 
    title='biblio.title', 
    applicant='biblio.applicants.0', 
    first_named_inventor='biblio.inventors.0').to_pandas()
foreign_df = foreign_df[foreign_df['country'] != 'US']
foreign_df

,country,appl_id,app_filing_date,pub_number,pub_date,title,applicant,first_named_inventor
0,CA,3071314,None,3071314,2019-02-07,NOVEL BATTERY SYSTEMS BASED ON LITHIUM DIFLUOR...,TESLA MOTORS CANADA ULC,"MA, LIN,"
1,CA,3013743,None,3013743,2019-12-20,DIOXAZOLONES AND NITRILE SULFITES AS ELECTROLY...,TESLA MOTORS CANADA ULC,"DAHN, JEFFERY RAYMOND,"
2,WO,2018000162,None,2019241869,2019-12-26,DIOXAZOLONES AND NITRILE SULFITES AS ELECTROLY...,TESLA MOTORS CANADA ULC,"DAHN, Jeffery,"
4,CA,3013596,None,3013596,2019-09-12,NOVEL BATTERY SYSTEMS BASED ON TWO-ADDITIVE EL...,TESLA MOTORS CANADA ULC,"DAHN, JEFFERY RAYMOND,"
5,WO,2018000164,None,2019173892,2019-09-19,NOVEL BATTERY SYSTEMS BASED ON TWO-ADDITIVE EL...,TESLA MOTORS CANADA ULC,"DAHN, Jeffery Raymond,"
6,WO,2018000163,None,2019173891,2019-09-19,NOVEL BATTERY SYSTEMS BASED ON TWO-ADDITIVE EL...,TESLA MOTORS CANADA ULC,"DAHN, Jeffery,"
9,CN,201780073009,None,110073170,2019-07-30,"OBSCURING, COLOR MATCHING, AND CAMOUFLAGING SO...",特斯拉公司,"迈克尔·皮里奥德,"
10,WO,2018051371,None,2019109167,2019-06-13,METHOD AND SYSTEM FOR DETERMINING CONCENTRATIO...,TESLA MOTORS CANADA ULC,"ELLIS, Leah Devorah,"
11,CA,2992228,None,2992228,2019-06-06,METHOD AND SYSTEM FOR DETERMINING CONCENTRATIO...,TESLA MOTORS CANADA ULC,"ELLIS, LEAH DEVORAH,"
13,WO,2018055745,None,2019025980,2019-02-07,NOVEL BATTERY SYSTEMS BASED ON LITHIUM DIFLUOR...,TESLA MOTORS CANADA ULC,"MA, Lin,"


And now we will take our results, and write to an excel file

In [10]:
writer = pd.ExcelWriter(f'{company_name}_portfolio.xlsx')
us_df.to_excel(writer, sheet_name='US Portfolio')
foreign_df.to_excel(writer, sheet_name='Foreign Portfolio')
writer.save()